In [1]:
import glob
import numpy as np
from PIL import Image

list_classes = [
 'Sony-NEX-7',
 'Motorola-X',
 'HTC-1-M7',
 'Samsung-Galaxy-Note3',
 'Motorola-Droid-Maxx',
 'iPhone-4s',
 'iPhone-6',
 'LG-Nexus-5x',
 'Samsung-Galaxy-S4',
 'Motorola-Nexus-6']

list_dict = {}
for i in range(10):
    key = list_classes[i]
    v = [0,0,0,0,0,0,0,0,0,0]
    v[i] = 1
    list_dict[key] = v
print(list_dict)

train_dir = '../input/train'
test_dir = '../input/test'
test_files = sorted(glob.glob(test_dir+'/*'))
train_files = sorted(glob.glob(train_dir+'/*/*'))
train_data_cnt = len(train_files)


def get_img(img_path,crop=224,train_flag = True):
    # read and resize
    im_array = np.array(Image.open((img_path)), dtype="uint8")
    pil_im = Image.fromarray(im_array)
    # new_array = np.array(pil_im.resize((512, 512)))
    
    # center crop
    center_x = im_array.shape[1] // 2
    half_crop = crop // 2
    if train_flag:
        rnd1 = np.random.randint(100) - 50
    else:
        rnd1 = 0
    a,b = center_x + rnd1 - half_crop, center_x + rnd1 + half_crop
    final_img = im_array[a:b,a:b,:]
    return final_img/255.0

def get_y(img_path):
    n = img_path.split('/')[-2]
    return list_dict[n]

{'Motorola-Nexus-6': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1], 'Motorola-X': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], 'iPhone-6': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0], 'Samsung-Galaxy-S4': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], 'HTC-1-M7': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], 'Motorola-Droid-Maxx': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0], 'iPhone-4s': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], 'Samsung-Galaxy-Note3': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0], 'Sony-NEX-7': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'LG-Nexus-5x': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]}


In [2]:
BATCH_SIZE = 48

def data_gen(file_list, batch_size=BATCH_SIZE, train_flag = True):
    curr_idx = 0
    data_cnt = len(file_list)
    while True:
        if curr_idx + batch_size > data_cnt:
            start_idx = data_cnt-batch_size
            end_idx = data_cnt
            curr_idx = 0
        else:
            start_idx = curr_idx
            end_idx = curr_idx + batch_size
            curr_idx += batch_size
        curr_fl = file_list[start_idx:end_idx]
        curr_x = np.array([get_img(p,224,train_flag) for p in curr_fl],dtype='float32')
        curr_y = np.array([get_y(p) for p in curr_fl])
        yield curr_x,curr_y

from sklearn.utils import shuffle
train_files = shuffle(train_files,random_state=42)
train_ratio = 0.9
train_cnt = int(train_data_cnt*train_ratio)
train_fl = train_files[:train_cnt]
valid_fl = train_files[train_cnt:]

train_gen = data_gen(train_fl, BATCH_SIZE, True)
valid_gen = data_gen(valid_fl, BATCH_SIZE, False)
train_step = train_cnt//BATCH_SIZE
valid_step = (train_data_cnt - train_cnt)//BATCH_SIZE

# test
for x,y in train_gen:
    print(x.shape,y.shape)
    print(x.dtype)
    print(y[:3])
    break
    

(48, 224, 224, 3) (48, 10)
float32
[[0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0]]


In [3]:
# def model
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam


def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()
    model.add(Conv2D(32, (3, 3),input_shape=(224, 224, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Conv2D(32, (3, 3), activation='relu', strides=1))
    model.add(Conv2D(32, (3, 3), activation='relu', strides=2))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(Conv2D(64, (3, 3), activation='relu', strides=2))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(10, activation='softmax'))
    return model
print('model model')
tmp_m = create_model()
tmp_m.summary()

Using TensorFlow backend.


model model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 109, 109, 32)      9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 54, 54, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 27, 27, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 25, 25, 64)        18496     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 12, 12, 64)        36928    

In [4]:
model = create_model()
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])
model_p = 'best_m.h5'
model_chk = ModelCheckpoint(filepath=model_p, monitor='val_acc', save_best_only=True, verbose=1)
model.fit_generator(train_gen,
          steps_per_epoch = train_step,
          epochs=50,
          validation_data = valid_gen,
          validation_steps = valid_step,
          callbacks=[model_chk]
         )

Epoch 1/50
51/51 [==============================] - 367s 7s/step - loss: 2.3005 - acc: 0.0980 - val_loss: 2.3028 - val_acc: 0.1000
Epoch 2/50
51/51 [==============================] - 323s 6s/step - loss: 2.2876 - acc: 0.1152 - val_loss: 2.2222 - val_acc: 0.1542
Epoch 3/50
51/51 [==============================] - 322s 6s/step - loss: 2.2529 - acc: 0.1409 - val_loss: 2.2051 - val_acc: 0.1708
Epoch 4/50
51/51 [==============================] - 322s 6s/step - loss: 2.2323 - acc: 0.1552 - val_loss: 2.1938 - val_acc: 0.1583
Epoch 5/50
51/51 [==============================] - 323s 6s/step - loss: 2.2036 - acc: 0.1634 - val_loss: 2.1577 - val_acc: 0.2042
Epoch 6/50
51/51 [==============================] - 322s 6s/step - loss: 2.1737 - acc: 0.1855 - val_loss: 2.1701 - val_acc: 0.1833
Epoch 7/50
51/51 [==============================] - 322s 6s/step - loss: 2.1616 - acc: 0.1752 - val_loss: 2.1097 - val_acc: 0.2042
Epoch 8/50
51/51 [==============================] - 317s 6s/step - loss: 2.1326 - a

Epoch 32/50
51/51 [==============================] - 323s 6s/step - loss: 1.6192 - acc: 0.4020 - val_loss: 1.6461 - val_acc: 0.4500
Epoch 33/50
51/51 [==============================] - 322s 6s/step - loss: 1.6494 - acc: 0.4109 - val_loss: 1.8251 - val_acc: 0.4000
Epoch 34/50
51/51 [==============================] - 322s 6s/step - loss: 1.6250 - acc: 0.4065 - val_loss: 1.8865 - val_acc: 0.3750
Epoch 35/50
51/51 [==============================] - 322s 6s/step - loss: 1.6304 - acc: 0.3971 - val_loss: 1.7687 - val_acc: 0.4083
Epoch 36/50
51/51 [==============================] - 322s 6s/step - loss: 1.5538 - acc: 0.4477 - val_loss: 1.7093 - val_acc: 0.4375
Epoch 37/50
51/51 [==============================] - 322s 6s/step - loss: 1.5667 - acc: 0.4416 - val_loss: 1.5296 - val_acc: 0.4417
Epoch 38/50
51/51 [==============================] - 322s 6s/step - loss: 1.5241 - acc: 0.4379 - val_loss: 1.6123 - val_acc: 0.4417
Epoch 39/50
51/51 [==============================] - 322s 6s/step - loss: 1.

In [6]:
best_model = load_model(model_p)
test_y = []
for img_p in test_files:
    tmp_x = get_img(img_p,224,False)
    tmp_y = best_model.predict(np.array([tmp_x]))[0]
    test_y.append(tmp_y)

test_y = np.array(test_y)
print(test_y[:5])

[[  1.99638173e-01   9.70054269e-02   2.34669298e-01   5.55319563e-02
    1.39542803e-01   2.21688636e-02   1.49013875e-02   8.23963135e-02
    4.62826006e-02   1.07863203e-01]
 [  7.66278774e-09   1.26324534e-01   2.93884100e-03   2.69204262e-03
    4.06231811e-06   3.64735443e-03   1.54590627e-04   6.85762987e-02
    2.44139712e-02   7.71248221e-01]
 [  2.74915433e-08   2.25176737e-02   6.17560744e-03   1.97880026e-02
    1.19570575e-06   2.95941476e-02   9.12524993e-04   5.27959049e-01
    8.03619996e-03   3.85015488e-01]
 [  7.95043491e-07   7.80305341e-02   1.96689798e-04   1.77442178e-01
    2.27470417e-04   6.28497303e-02   3.41156758e-02   4.74554822e-02
    4.80016887e-01   1.19664550e-01]
 [  1.21743518e-11   9.86096710e-02   1.13044862e-05   4.59026635e-01
    2.68367586e-08   6.97050593e-04   6.62197053e-05   1.43033072e-01
    1.25342518e-01   1.73213527e-01]]


In [11]:
import pandas as pd
y_res = np.argmax(test_y,axis=1)
y_res = [list_classes[i] for i in y_res]
df = pd.read_csv('../input/sample_submission.csv')
print(df.head())
f_name = [p.split('/')[-1] for p in test_files]
df['fname'] = f_name
df['camera'] = y_res
print(df.head())
df.to_csv('../results/s_cnn_224.csv',index=False)

                   fname    camera
0  img_0002a04_manip.tif  iPhone-6
1  img_001e31c_unalt.tif  iPhone-6
2  img_00275cf_manip.tif  iPhone-6
3  img_0034113_unalt.tif  iPhone-6
4  img_00344b7_unalt.tif  iPhone-6
                   fname                camera
0  img_0002a04_manip.tif              HTC-1-M7
1  img_001e31c_unalt.tif      Motorola-Nexus-6
2  img_00275cf_manip.tif           LG-Nexus-5x
3  img_0034113_unalt.tif     Samsung-Galaxy-S4
4  img_00344b7_unalt.tif  Samsung-Galaxy-Note3
